#### Imports

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
import time
import numpy as np, heapq
import pandas as pd
import codecs
import lxml
import csv
import os.path
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
import nltk
from collections import defaultdict
import pickle
from tqdm import tqdm
import math
import time
from pathlib import Path
from joblib import Parallel, delayed # parallel processing
import functions
from numpy.linalg import norm
import pandas as pd
import numpy as np
import ast

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Constant variable

In [2]:
urls_txt = "https.txt"
folder = 'Task 1.2/dataset'
folder2 = 'Task 1.3/anime_tsv'
headers = {
    'user-agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36",
    'accept': "image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8",
    'referer': "https://myanimelist.net/"
} 
save_path = 'simplePath'
trfile = 'file.csv'

## 1 Data Collection

### 1.1 Get the list of animes

In [4]:
c = 0
reg = r'href=\"https://[\w\W]*" id'
htt = list()
for i in range(400):
    cnt = requests.get("https://myanimelist.net/topanime.php?limit="+str(i*50))
    soup = BeautifulSoup(cnt.content, features="lxml")
    links = soup.find_all('h3')
    for l in range(len(links)-3):
        li = links[l].find('a')
        m = re.findall(reg, str(li))
        s = m[0][6:][:-4]+'\n'
        htt.append(s)
f = open("urls_txt", 'w')
for i in htt:
    f.write(i)
f.close

<function TextIOWrapper.close()>

### 1.2 Crawl animes

htmls_by_urls

In [ ]:
functions.htmls_by_urls(urls_txt, folder)

### 1.3 Parse downloaded pages

In [7]:
# find all the path to the file
alarr = functions.getHTML(folder)

In [8]:
reg1 = r'\n[\w\W]*to'
reg2 = r'to[\w\W]*\n'
animeTitle = []
animeTypes = []
animeNumEpisode = []
releaseDate = []
endDate = []
animeNumMembers = []
animeScore = []
animeUsers = []
animeRank = []
animePopularity = []
animeDescription = []
animeRelated = []
animeCharacters = []
animeVoices = []
animeStaff = [] 
df = pd.DataFrame(
[animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff], 
index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
).T
bs = 9
for fiName in range(len(alarr)):
    # anime titles
    path = folder+'/'+alarr[fiName]
    #print(path)
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')
    animeTitle.append(soup.find_all('strong')[0].contents[0])
    
    # left sied of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "spaceit_pad"})
    for div in divs:
        spans = div.find_all("span")
        for span in spans:
            # anime types
            if span.contents[0] == 'Type:':
                tr = div.find_all('a')
                if len(tr) ==0:
                    animeTypes.append(div.contents[2][2:])
                else:
                    animeTypes.append(div.find_all('a')[0].contents[0])
            # anime number of episodes
            if span.contents[0] == 'Episodes:':
                try :
                    animeNumEpisode.append(int(div.contents[2]))
                except:
                    animeNumEpisode.append(0)
            # anime release and end dates
            if span.contents[0] == 'Aired:':
                if 'Not available' in div.contents[2]:
                    releaseDate.append('?')
                    endDate.append('?')
                elif 'to' in div.contents[2]:                    
                    if "?" in re.findall(reg2, str(div.contents[2]))[0][3:-1]:
                        endDate.append("?")
                    else:
                        endDate.append(pd.to_datetime(re.findall(reg2, str(div.contents[2]))[0][3:-1]))
                    if "?" in re.findall(reg1, str(div.contents[2]))[0][3:-1]:
                        releaseDate.append("?")
                    else:
                        releaseDate.append(pd.to_datetime(re.findall(reg1, str(div.contents[2]))[0][2:-3]))
                else:
                    releaseDate.append(pd.to_datetime(div.contents[2][2:-2]))
                    endDate.append('-')
                    
    # middle side of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "stats-block po-r clearfix"})
    for div in divs:
        # anime number of members
        members = div.find_all("span", {"class": "numbers members"})
        animeNumMembers.append(int(members[0].contents[1].contents[0].replace(',', '')))
        # anime score
        score = div.find_all("div", {"class": "score-label score-"+str(bs)})
        while len(score) == 0:
            bs -= 1
            score = div.find_all("div", {"class": "score-label score-"+str(bs)})
            if bs == -1:
                bs = 10
                score = div.find_all("div", {"class": "score-label score-na"})
        try:        
            animeScore.append(float(score[0].contents[0]))
            # anime number of users
            users = div.find_all("div", {"class": "fl-l score"})
            animeUsers.append(int(users[0]['data-user'][:-6].replace(',', '')))
        except:
            animeScore.append(None)
            animeUsers.append(0)

        # anime rank
        rank = div.find_all("span", {"class": "numbers ranked"})
        try:
            animeRank.append(int(rank[0].contents[1].contents[0][1:]))
        except:
            animeRank.append(-1)
        # anime popularity
        popularity = div.find_all("span", {"class": "numbers popularity"})
        animePopularity.append(int(popularity[0].contents[1].contents[0][1:]))
        
    # anime description
    description = soup.find_all("p", {"itemprop": "description"})
    for br in description[0].find_all("br"):
        br.replace_with("\n")
    animeDescription.append(description[0].contents)
    
    # related animes 
    related = soup.find_all("table", {"class": "anime_detail_related_anime"})
    x = []
    y = []
    for tr in related:
        td = tr.find_all("td")
        for i in range(0, len(td), 2):
            x.append(td[i].contents[0])
            try:
                t = td[i+1].find_all("a")
                y.append(t[0].contents[0])
            except:
                y.append('NA')
            
        animeRelated.append('\n'.join([f'{x} {y}' for x, y in dict(zip(x, y)).items()]).split('\n'))
        if(len(animeRelated)<len(animeDescription)):
            animeRelated.append("None")
    try:    
        # anime characters and voices
        characters = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        chars = characters[0].find_all("h3", {"class": "h3_characters_voice_actors"})
        x = []
        y = []
        voices = characters[0].find_all("td", {"class": "va-t ar pl4 pr4"})
        for i in chars:
            x.append(i.contents[0].contents[0])
        for i in voices:
            y.append(i.contents[1].contents[0])
        animeCharacters.append(x)
        animeVoices.append(y)
    except:
        animeCharacters.append("NA")
        animeVoices.append("NA")
    
    # anime staff
    try:
        staff = soup.find_all("div", {"class": "detail-characters-list clearfix"})
        staff = staff[1].find_all("td")
        x = []
        y = []
        for i in range(1, len(staff), 2):
            x.append(staff[i].contents[1].contents[0])
            y.append(staff[i].find_all("small")[0].contents[0])
        animeStaff.append([list(i) for i in list(zip(x,y))])
    except:
        animeStaff.append("NA")
        
        
    # Following lines are made to save memory by storing the current info in a pd instead of the array    
    if ((fiName+1)%500) == 0:
        dftr = pd.DataFrame(
        [animeTitle.copy(), animeTypes.copy(), animeNumEpisode.copy(), releaseDate.copy(), endDate.copy(), animeNumMembers.copy(), animeScore.copy(), animeUsers.copy(), animeRank.copy(), animePopularity.copy(), animeDescription.copy(), animeRelated.copy(), animeCharacters.copy(), animeVoices.copy(), animeStaff.copy()], 
        index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
        ).T
        df = pd.concat([df, dftr])
        df.to_csv(trfile, index=False)
        del df, dftr
        animeTitle = []
        animeTypes = []
        animeNumEpisode = []
        releaseDate = []
        endDate = []
        animeNumMembers = []
        animeScore = []
        animeUsers = []
        animeRank = []
        animePopularity = []
        animeDescription = []
        animeRelated = []
        animeCharacters = []
        animeVoices = []
        animeStaff = [] 
        df = pd.read_csv(trfile)
        os.remove(trfile)

In [9]:
dftr = pd.DataFrame(
[animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff], 
index=['Title', 'Type', 'Episodes','Release date', 'End date', 'Members', 'Score', 'Users', 'Rank', 'Popularity', 'Description', 'Related', 'Characters', 'Voices', 'Staff']
).T
df = pd.concat([df, dftr])
df.to_csv(trfile, index=False)
del df, dftr, animeTitle, animeTypes, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff
df = pd.read_csv(trfile)

In [17]:
df.head()
df.shape

(19053, 15)

In [11]:
#For each row I created a tsv file with the value of each anime 
for i in range(len(df)):
    with open(folder2+'/anime_'+str(i)+'.tsv', 'wt') as file:
        tsv_writer = csv.writer(file, delimiter='\t')
        tsv_writer.writerow([x for x in df.columns]) #the header row
        tsv_writer.writerow(x for x in df.iloc[i]) #the value under each columns
        

## 2 Search Engine

In [3]:
dataset = functions.retriveTSV(folder2)

# We add the different url as it is ask for the search engine
with open('urls_txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
list_url = [line.strip() for line in lines]
dataset['Url'] = list_url
dataset.head()

,Title,Type,Episodes,Release date,End date,Members,Score,Users,Rank,Popularity,Description,Related,Characters,Voices,Staff,Url
0,Fullmetal Alchemist: Brotherhood,TV,64,2009-04-05,2010-07-04 00:00:00,2676639,9.16,1622384,1,3,"[""After a horrific alchemy experiment goes wro...","['Adaptation: Fullmetal Alchemist', 'Alternati...","['Elric, Edward', 'Elric, Alphonse', 'Mustang,...","['Park, Romi', 'Kugimiya, Rie', 'Miki, Shinich...","[['Cook, Justin', 'Producer'], ['Yonai, Norito...",https://myanimelist.net/anime/5114/Fullmetal_A...
1,Gintama: The Final,Movie,1,2021-01-08,-,79486,9.00,29979,10,1924,"['New ', <i>Gintama</i>, ' movie.']","['Adaptation: Gintama', 'Prequel: Gintama.: Sh...","['Sakata, Gintoki', 'Kagura', 'Shimura, Shinpa...","['Sugita, Tomokazu', 'Ishida, Akira', 'Hino, S...","[['Fujita, Youichi', 'Director'], ['Miyawaki, ...",https://myanimelist.net/anime/28977/Gintama°
2,Shingeki no Kyojin: The Final Season,TV,16,2020-12-07,2021-03-29 00:00:00,1346547,8.91,791929,19,57,"[""Gabi Braun and Falco Grice have been trainin...","['Adaptation: Shingeki no Kyojin', 'Prequel: S...","['Yeager, Eren', 'Ackerman, Mikasa', 'Arlert, ...","['Kaji, Yuki', 'Ishikawa, Yui', 'Inoue, Marina...","[['Hayashi, Yuuichirou', 'Director, Storyboard...",https://myanimelist.net/anime/38524/Shingeki_n...
3,Uchuu Senkan Yamato 2199: Tsuioku no Koukai,Special,1,2014-10-11,-,2191,6.73,529,4950,9205,['Recap of Uchuu Senkan Yamato 2199.'],['Full story: Uchuu Senkan Yamato 2199'],"['Kodai, Susumu', 'Niimi, Kaoru', 'Kiryuu, Mik...","['Nakamura, Eriko']","[['Kato, Takao', 'Director'], ['Imanishi, Taka...",https://myanimelist.net/anime/9253/Steins_Gate
4,Ikkitousen: Xtreme Xecutor Specials,Special,6,2010-06-25 00:00:00,2010-11-25 00:00:00,12848,6.56,4732,5805,4788,['Ikkitousen: Xtreme Xecutor Blu-ray and DVD s...,['Parent story: Ikkitousen: Xtreme Xecutor'],"[""Kan'u, Unchou"", 'Ryomou, Shimei', 'Sonsaku, ...","['Nabatame, Hitomi', 'Kaida, Yuuko', 'Asano, M...","[['Gotou, Junji', 'Chief Animation Director, C...",https://myanimelist.net/anime/42938/Fruits_Bas...


In [4]:
dataset.shape

(19053, 16)

### 2.0 Cleaning

In [5]:
dataset['Description'][0]

'["After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse\'s body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse\'s soul in the physical realm by binding it to a hulking suit of armor.", \'\\n\', \'\\n\', \'\\n\', \'\\r\\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher\\\'s Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent

In [6]:
functions.clean(functions.tokenize(dataset['Description'][0]))

['horrif',
 'alchemi',
 'experi',
 'goe',
 'wrong',
 'elric',
 'household',
 'brother',
 'edward',
 'alphons',
 'left',
 'catastroph',
 'new',
 'realiti',
 'ignor',
 'alchem',
 'principl',
 'ban',
 'human',
 'transmut',
 'boy',
 'attempt',
 'bring',
 'recent',
 'deceas',
 'mother',
 'back',
 'life',
 'instead',
 'suffer',
 'brutal',
 'person',
 'loss',
 'alphons',
 'bodi',
 'disintegr',
 'edward',
 'lost',
 'leg',
 'sacrif',
 'arm',
 'keep',
 'alphons',
 'soul',
 'physic',
 'realm',
 'bind',
 'hulk',
 'suit',
 'armor',
 'nthe',
 'brother',
 'rescu',
 'neighbor',
 'pinako',
 'rockbel',
 'granddaught',
 'winri',
 'known',
 'bio',
 'mechan',
 'engin',
 'prodigi',
 'winri',
 'creat',
 'prosthet',
 'limb',
 'edward',
 'util',
 'automail',
 'tough',
 'versatil',
 'metal',
 'use',
 'robot',
 'combat',
 'armor',
 'year',
 'train',
 'elric',
 'brother',
 'set',
 'quest',
 'restor',
 'bodi',
 'locat',
 'philosoph',
 'stone',
 'power',
 'gem',
 'allow',
 'alchemist',
 'defi',
 'tradit',
 'law',
 

In [13]:
datasetClean = defaultdict()
for i in range(len(dataset)):
    datasetClean[i]=(functions.clean_fast(functions.tokenize((dataset['Description'][i]))))
# We save dictionaries as pkl
descriptionsDataset = open("descriptionsDataset.pkl", "wb")
pickle.dump(datasetClean, descriptionsDataset)
descriptionsDataset.close()

### 2.1 Conjunctive query

#### 2.1.1 Create your index!

### Dictionaries

- the first dictionary <code>word_2_id</code> maps word to word identification integer

- the inverted index dictionary <code>id_2_anime</code> maps word identification integer to list of indexes (main dataset indexes) of anime whose cleaned description contains the word identified by the integer

In [7]:
word_2_id0, id_2_anime0 = functions.dictionaries(dataset)

100%|██████████| 19053/19053 [00:30<00:00, 628.83it/s]


#### 2.1.2 Execute the query

In [8]:
q = functions.search_engine("saiyan race")
q

[1300, 8854, 9451, 14816]

In [9]:
dfq = dataset.iloc[q, [0, 10, -1]]
dfq.reset_index(drop=True, inplace=True)
dfq

,Title,Description,Url
0,Dragon Ball Z Special 1: Tatta Hitori no Saish...,"[""Bardock, Son Goku's father, is a low-ranking...",https://myanimelist.net/anime/11737/Ao_no_Exor...
1,Dragon Ball Z,"[""Five years after winning the World Martial A...",https://myanimelist.net/anime/37896/Ling_Yu_6t...
2,Dragon Ball Super: Broly,"[""Forty-one years ago on Planet Vegeta, home o...",https://myanimelist.net/anime/35952/Devilman_M...
3,Dragon Ball Kai,"[""Five years after the events of Dragon Ball, ...",https://myanimelist.net/anime/48287/Allegseuui...


### 2.2 Conjunctive query & Ranking score

For the second search engine, given a query, we want to get the top-k (the choice of k it's up to you!) documents related to the query. In particular:

- Find all the documents that contains all the words in the query.
- Sort them by their similarity with the query.
- Return in output k documents, or all the documents with non-zero similarity with the query when the results are less than k. You must use a heap data structure (you can use Python libraries) for maintaining the top-k documents.
- To solve this task, you will have to use the tfIdf score, and the Cosine similarity. The field to consider it is still the synopsis.

In [14]:
import pickle


with open('id_2_anime.pkl', 'rb') as f:
    firstDictionary = pickle.load(f)
    
with open('word_2_id.pkl', 'rb') as f:
    dataWord = pickle.load(f)
    
with open('descriptionsDataset.pkl', 'rb') as f:
    dataDescriptionClean = pickle.load(f)

#### 2.2.1 Inverted index

In [15]:
lengthDic = len(firstDictionary)

In [16]:
tfidf_invIndex = defaultdict() #starting a dictiornary

dataWordList = list(dataWord.copy()) #copy of the dictionary which contains the words
tfidf_invIndex = firstDictionary.copy() #copy of the dictionary which containts the documents where each word of dataword is present
dataDescription = dataDescriptionClean.copy()

In [17]:
for i in tqdm(range(1,len(firstDictionary))):
    lenghtTerm = len(firstDictionary[i]) #lenght of a single term_id_i
    for j in range(len(firstDictionary[i])):
        synopsis = dataDescription[firstDictionary[i][j]] #the description that contains the word
        wordsDocument = len(synopsis) #the lenght
        numberOfOccurence = functions.number_occurence(synopsis, dataWordList[i])
        tfidf_invIndex[i][j]=(firstDictionary[i][j], functions.calculate_TfIdf(lengthDic, lenghtTerm, numberOfOccurence, wordsDocument)) #return ( id-document, tfIdf_{term,documenti})

100%|██████████| 39597/39597 [00:05<00:00, 7408.91it/s] 


#### 2.2.2 Execute the query

In [19]:
searchWord = (functions.clean_fast(functions.tokenize(('saiyan race'))))
scoreHeap, k = [], 10
heapq.heapify(scoreHeap)
q =  np.zeros((len(dataWord)))

for word in searchWord:
    for key, value in dataWord.items():
        if word == key:
            print(word, key)
            q[value] +=1 #fill the query q
normq = norm(q, ord=2)
normq

saiyan saiyan
race race


1.4142135623730951

In [20]:
d = np.zeros((len(dataset), len(dataWord)))

for i in tqdm(range(len(dataset))):
    for j in range(1,len(q)):
        if (q[j] != 0):
            for k in range(len(tfidf_invIndex[j])):
                if i == tfidf_invIndex[j][k][0]:
                    d[i][j] = (tfidf_invIndex[j][k][1])

100%|██████████| 19053/19053 [06:02<00:00, 52.61it/s]


In [21]:
for i in tqdm(range(len(d))):
    my_sum = 0
    normd = norm(d[i])
    my_sum = np.sum([q[j]*d[i][j] for j in range(len(q)) if q[j] != 0])
    if not math.isnan(my_sum/(normq*normd)):
        heapq.heappush(scoreHeap,(i,my_sum/(normq*normd)))

    if len(scoreHeap)==k:
        heapq.heappop(scoreHeap)
    scoreHeap.sort(key=lambda x:x[1])

  0%|          | 0/19053 [00:00<?, ?it/s]/home/ec2-user/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """
100%|██████████| 19053/19053 [05:44<00:00, 55.33it/s]


In [22]:
scoreHeap.sort(key=lambda x:x[1], reverse=True)
out = []
for x in range(10):
    id = scoreHeap[x][0]
    title = dataset['Title'][id]
    des = dataset['Description'][id]
    url = ""
    cosine = scoreHeap[x][1]
    out.append((id,title,des,url,cosine))
    print(out)

[(1300, 'Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen', '["Bardock, Son Goku\'s father, is a low-ranking Saiyan soldier who was given the power to see into the future by the last remaining alien on a planet he just destroyed. He witnesses the destruction of his race and must now do his best to stop Frieza\'s impending massacre.", \'\\n\', \'\\n\', \'\\n\', \'\\r\\n(Source: ANN)\']', '', 0.9670745372626464)]
[(1300, 'Dragon Ball Z Special 1: Tatta Hitori no Saishuu Kessen', '["Bardock, Son Goku\'s father, is a low-ranking Saiyan soldier who was given the power to see into the future by the last remaining alien on a planet he just destroyed. He witnesses the destruction of his race and must now do his best to stop Frieza\'s impending massacre.", \'\\n\', \'\\n\', \'\\n\', \'\\r\\n(Source: ANN)\']', '', 0.9670745372626464), (9451, 'Dragon Ball Super: Broly', '["Forty-one years ago on Planet Vegeta, home of the infamous Saiyan warrior race, King Vegeta noticed a baby named Broly

# Task 3. Define a new score!
### What kind of a score we are defining? Let's answer this question.
To make a score, we need some kind of an information. Because we are taking a query as an input, we can ask people who are writing the query to give some additional information. As a person who watches anime a lot, I know some people might be too specific with their query: some like longer animes (>100 episodes), some like specific producers' works, some of them even watch the anime only because of the seiyuu (voice actor). But most of the people don't care about stuff like that, so we need to have an option to skip some questions in the query. 

Let's with adding a new column for our newly defined score:

In [ ]:
df = functions.retriveTSV(trfile)
df.head()

In [ ]:
score = np.zeros(len(df))
df['NewScore'] = score
df.head()

### The function below asks the requester to give their query. It contains:
- text query
- type of an anime
- preferred number of episodes
- how old the anime should be
- popularity of the anime
- how many related works it has
- preferred voice actors
- preferred staff members

Every question might be skipped if the requester has no preference. <br />
This function is define in function.py

### The function _new_score_ takes the dataframe and the query inputs to calculate the new score.
The algorithm is pretty simple. It takes the best matches by the query and gives them some kind of a score. After summing the score of each part, we return the updated copy of our dataframe.<br />
This function is define in function.py

In [ ]:
Q, Type, NumOfEpisodes, AgeOfAnime, Popularity, NumOfRelated, Voices, Staff = query_details()

    #***********************************************************#
    #    d = full dataset                                       #
    #    d1 = query subsection                                  #
    #    Type = ['TV', 'Movie', 'Special']                      #
    #    NumOfEpisodes = n/Episodes                             #
    #    AgeOfAnime = ['new', 'old', 'moderate']                #
    #    Popularity = ['popular', 'moderate', 'not popular']    #
    #    NumOfRelated = ['single', 'few seasons', 'many parts'] #
    #    Voices = [Surname, Name]                               #
    #    Staff = [Surname, Name]                                #
    #***********************************************************#
    
# Getting a subset of the dataframe by searching the animes that satisfy the query
query = search_engine(Q)
q_df = df.iloc[query].copy()
q_df.head()

## The results of Task 3
The result is the dataframe sorted by the newly defined score. The score means the closest match to the query, the anime that the person should watch the most by his query. Or the person was just searching the specific anime that he forget the name of. Anyway, here is the result:

In [ ]:
new_df = functions.new_score(df, q_df, Type, NumOfEpisodes, AgeOfAnime, Popularity, NumOfRelated, Voices, Staff).sort_values('NewScore', ascending=False)
new_df.sort_values('NewScore', ascending=False)

## 4 Understanding the anime's reviews

In a first time we will retrive the first 4 reviews (We don't do on 5 first as ask as the html file contains only the 4 first) and execute a sentiment analysis on these review for this we use the prebuild model in nltk library.<br />
the review will be score by the compound who is an agregation of the score between -1 and 1 (1 is the max value -1 is the min)

In [ ]:
sia = SentimentIntensityAnalyzer()
df = functions.retriveTSV(trfile)
popularity_Review_Based = list()
alarr = functions.getHTML(folder)
for fiName in range(len(alarr)):
    # anime titles
    path = folder+'/'+alarr[fiName]
    #print(path)
    file = codecs.open(path, "r", "utf-8")
    soup = BeautifulSoup(file, 'html.parser')    
    # left sied of the html------------------------------------------------------------
    divs = soup.find_all("div", {"class": "spaceit textReadability word-break pt8 mt8"})
    st = ""
    for i in divs:
        st += i.contents[2][:500]
    if st == "":
        popularity_Review_Based.append(-2)
    else:
        popularity_Review_Based.append(sia.polarity_scores(st)["compound"])
df.insert(0,'popularityReviewBased', popularity_Review_Based)

In a segond time we will select the row who contains the best score and display it

In [ ]:
df[df['popularityReviewBased'].max() == df['popularityReviewBased']].head()

16778    Ganbarist! Shun
Name: Title, dtype: object

In the following part of the code we try to determine if there is an correlation between the number of episod and the sentiment analysis. For doing that we will in the first time remove all the data who have not more than 1 episod (this would indcated movie) and all the anime who don't have review.<br />
In a segond time we will perform a linear regretion on one part of the data (trainingSet) for trying to see if there is a link and will compare 

In [ ]:
isSerie = df['Episodes'] > 1
isReview = df['popularityReviewBased'] != -2
data = df[['popularityReviewBased','Episodes']][isSerie & isReview]

,popularityReviewBased,Episodes
0,0.9892,64
2,0.9859,12
3,0.9833,22
5,0.9312,24
6,0.9662,201


In [ ]:
X = data["popularityReviewBased"]
Y = data['Episodes']

data = data.sample(frac=1).reset_index(drop=True) # shuffle data
percentage = round(len(data.index)*2/3)
X_train = (pd.DataFrame(data=X).iloc[:percentage]).to_numpy()
Y_train = (pd.DataFrame(data=Y).iloc[:percentage]).to_numpy()

X_test = (pd.DataFrame(data=X).iloc[percentage:]).to_numpy()
Y_test = (pd.DataFrame(data=Y).iloc[percentage:]).to_numpy()

                       popularityReviewBased  Episodes
popularityReviewBased               1.000000  0.064861
Episodes                            0.064861  1.000000


In [ ]:
from sklearn import linear_model
reg = linear_model.LinearRegression()
reg.fit(X_train.reshape(-1, 1),Y_train)
Y_result = reg.predict(X_test)
diff = 0
for i in range(len(Y_result)):
    diff += abs(Y_result[i] - Y_test[i])
print(diff/len(Y_result))

[[3.79190313]]
[14.11944127]


## 5 Algorithmic question